In [16]:
from IPython.display import display, HTML
import pandas as pd
import numpy

In [17]:
text = open("../Lovers slash list cleaned.csv").read()

In [18]:
shiplist_raw = [line for line in text.split("\n")[1:] if "/" in line]

In [19]:
from collections import Counter, defaultdict

In [20]:
shiplist = sorted(Counter(shiplist_raw).items(), key=lambda x:x[1], reverse=True)

In [21]:
char_in_pair_freq = defaultdict(int)
for pair, score in shiplist:
    try: 
        left_name, right_name = pair.split("/")
    except:
        print(f"špatný počet lomítek v {pair}")
    char_in_pair_freq[left_name] += score
    char_in_pair_freq[right_name] += score
sorted_char_in_pairs = sorted(Counter(char_in_pair_freq).items(), key=lambda x:x[1], reverse=True)

In [22]:
print("Nechme si jen jména s frekvencí větší než 100, zbyde nám jmen:", len([(char, freq) for char, freq in sorted_char_in_pairs if freq >= 100]))


Nechme si jen jména s frekvencí větší než 100, zbyde nám jmen: 38


In [23]:
filtered_char_in_ship_38_best = [(char, freq) for char, freq in sorted_char_in_pairs if freq >= 49]

In [24]:
empty = numpy.zeros((len(filtered_char_in_ship_38_best), len(filtered_char_in_ship_38_best)), dtype=int)

In [25]:
filtered_shiplist = []
filtered_char_without_freq = [ship for ship, _ in filtered_char_in_ship_38_best]

def index(name):
    """Harry je na nulté pozici, když mu zadám Harry, vrátí 0."""
    return filtered_char_without_freq.index(name)

for ship, count in shiplist:
    left_name, right_name = ship.split("/")
    if left_name in filtered_char_without_freq and right_name in filtered_char_without_freq:
        filtered_shiplist.append((ship, count))
        if left_name == right_name:
            print("Sebevztah", left_name, count)
            empty[index(left_name), index(right_name)] = count
        else:
            empty[index(left_name), index(right_name)] = count
            empty[index(right_name), index(left_name)] = count
        
        
len(filtered_shiplist)

Sebevztah Original_Male_Character 7
Sebevztah Original_Character 4
Sebevztah Lily_Evans_Potter 1
Sebevztah Tom_Riddle 1
Sebevztah Gellert_Grindelwald 1


286

In [26]:
from pandas import DataFrame

In [27]:
df = DataFrame(empty)

keep = numpy.triu(num.ones(df.shape)).astype('bool').reshape(df.size)


NameError: name 'num' is not defined

In [28]:
body = []
beautify_chars = [name.replace("_", "&nbsp;") for name in filtered_char_without_freq]
for row_number, name in enumerate(beautify_chars):
    start = f"<tr>"
    start += f"<td>{name}</td>"
    for col_number in range(len(beautify_chars)):
        value = empty[row_number, col_number]
        if col_number >= row_number:
            # obarvení podle hodnoty
            if value >= 500:
                start += f"<td style='background-color: #caffc4; width: 4ex'>{value}</td>"
            elif value >= 100:
                start += f"<td style='background-color: #c2e3bf; width: 4ex'>{value}</td>"
            elif value == 0:
                start += f"<td style='background-color: white; width: 4ex'>{value}</td>"
            else:
                start += f"<td style='background-color: #829487; width: 4ex'>{value}</td>"
        else:
            start += "<td></td>"
    start += "</tr>\n"
    body.append(start)

html_table = f"""
<table>
    <thead style="writing-mode: vertical-lr; text-orientation: use-glyph-orientation; text-align: right">
        <th></th><th>{"</th><th>".join(beautify_chars)}</th>
    </thead>
    {"".join(body)}
</table>
"""


In [29]:
HTML(html_table)

Harry Potter,0,7996,95,28,1262,0,7,2,27,8,767,13,5,557,1,21,0,155,2,12,0,1,11,0,43,59,11,60,11,50,0,8,2,7,4,2,34,34,67,0,0,0,0,21,2,4,11,0,0
Draco Malfoy,,0,8,5,49,0,0,1,0,4,11,3,26,6,0,0,0,148,3,2,1,0,0,17,71,13,3,17,7,43,2,0,3,4,1,0,11,20,4,0,0,0,0,3,0,1,1,0,0
Sirius Black,,,0,2430,98,0,0,0,0,3,2,0,0,0,0,0,0,1,0,196,0,0,0,1,0,24,1,0,0,4,0,28,0,1,0,0,2,0,0,0,1,0,0,0,0,1,3,0,0
Remus Lupin,,,,0,141,0,0,0,0,6,0,0,0,0,0,0,0,1,0,15,0,0,0,2,7,10,0,0,0,0,0,15,0,0,0,0,3,0,0,0,0,0,1,0,0,2,1,0,0
Severus Snape,,,,,0,0,0,7,13,4,15,0,0,15,0,0,0,14,0,64,0,0,0,0,9,72,1,0,0,2,0,20,0,0,0,0,6,0,0,0,3,0,0,0,0,37,6,0,0
Newt Scamander,,,,,,0,507,51,59,0,0,0,0,0,76,0,363,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,2
Original Percival Graves,,,,,,,0,56,3,0,0,0,0,0,413,0,72,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,5
Gellert Grindelwald,,,,,,,,1,852,0,2,0,0,0,32,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0
Albus Dumbledore,,,,,,,,,0,0,8,0,0,2,2,0,10,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,2,0,14
Hermione Granger,,,,,,,,,,0,1,0,0,0,0,62,0,17,69,0,168,282,0,0,0,0,0,0,0,0,79,0,0,0,0,67,0,0,0,38,2,10,8,0,6,0,2,1,0
Tom Riddle,,,,,,,,,,,1,0,0,1,1,3,0,2,0,2,0,0,0,0,0,2,0,0,1,0,0,2,0,1,0,0,3,0,1,0,0,0,0,0,0,0,1,0,0


In [30]:
frame = pd.DataFrame(data=empty, index=filtered_char_without_freq, columns=filtered_char_without_freq)

In [31]:
frame

,Harry_Potter,Draco_Malfoy,Sirius_Black,Remus_Lupin,Severus_Snape,Newt_Scamander,Original_Percival_Graves,Gellert_Grindelwald,Albus_Dumbledore,Hermione_Granger,...,Minerva_McGonagall,Lily_Evans_Potter,Lavender_Brown,Nymphadora_Tonks,George_Weasley,Cho_Chang,Kingsley_Shacklebolt,Original_Character,Parvati_Patil,Gellert_Grindelwald_pretending_Percival_Graves
Harry_Potter,0,7996,95,28,1262,0,7,2,27,8,...,0,0,0,0,21,2,4,11,0,0
Draco_Malfoy,7996,0,8,5,49,0,0,1,0,4,...,0,0,0,0,3,0,1,1,0,0
Sirius_Black,95,8,0,2430,98,0,0,0,0,3,...,0,1,0,0,0,0,1,3,0,0
Remus_Lupin,28,5,2430,0,141,0,0,0,0,6,...,0,0,0,1,0,0,2,1,0,0
Severus_Snape,1262,49,98,141,0,0,0,7,13,4,...,0,3,0,0,0,0,37,6,0,0
Newt_Scamander,0,0,0,0,0,0,507,51,59,0,...,0,0,0,0,0,0,0,0,0,2
Original_Percival_Graves,7,0,0,0,0,507,0,56,3,0,...,0,0,0,0,0,0,0,0,0,5
Gellert_Grindelwald,2,1,0,0,7,51,56,1,852,0,...,0,0,0,0,0,0,0,0,0,0
Albus_Dumbledore,27,0,0,0,13,59,3,852,0,0,...,0,0,0,0,0,0,0,2,0,14
Hermione_Granger,8,4,3,6,4,0,0,0,0,0,...,38,2,10,8,0,6,0,2,1,0


In [32]:
frame.to_csv("lovers_matrix_slash.csv")